In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
from tqdm import tqdm

In [2]:

def create_embeddings_in_batches(
    input_file, 
    output_file, 
    text_column, 
    model_name='all-MiniLM-L6-v2', 
    batch_size=64
):
    """
    Читает CSV, генерирует эмбеддинги батчами и сохраняет результат.
    """
    
    # 1. Определение устройства (GPU или CPU)
    device ='mps'# 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Используется устройство: {device}")

    # 2. Загрузка модели
    print(f"Загрузка модели {model_name}...")
    model = SentenceTransformer(model_name, device=device)

    # 3. Чтение данных
    print("Чтение CSV файла...")
    df = pd.read_csv(input_file)
    
    # Проверка на пустые значения в колонке с текстом
    if df[text_column].isnull().any():
        print(f"Внимание: найдены пустые значения в колонке '{text_column}'. Заменяем на пустую строку.")
        df[text_column] = df[text_column].fillna("")

    texts = df[text_column].tolist()
    total_rows = len(texts)
    
    # 4. Генерация эмбеддингов батчами
    all_embeddings = []
    
    print(f"Начинаем генерацию эмбеддингов (всего строк: {total_rows}, размер батча: {batch_size})...")
    
    # Цикл по батчам с прогресс-баром
    for i in tqdm(range(0, total_rows, batch_size)):
        batch_texts = texts[i : i + batch_size]
        
        # Генерируем эмбеддинги для текущего батча
        # convert_to_numpy=True возвращает numpy массивы
        embeddings = model.encode(batch_texts, convert_to_numpy=True, show_progress_bar=False)
        
        # Добавляем в общий список (можно сразу писать в файл, если данных ОЧЕНЬ много)
        all_embeddings.extend(embeddings)

    # 5. Добавление колонки в DataFrame
    df['embedding'] = list(all_embeddings)

    # 6. Сохранение
    # Рекомендую сохранять в Parquet или Pickle, так как CSV превратит массивы в строки "[0.1, 0.2...]"
    if output_file.endswith('.csv'):
        print("Сохранение в CSV (вектора будут строками)...")
        df.to_csv(output_file, index=False)
    elif output_file.endswith('.parquet'):
        print("Сохранение в Parquet (рекомендуется)...")
        df.to_parquet(output_file, index=False)
    elif output_file.endswith('.pkl'):
        print("Сохранение в Pickle...")
        df.to_pickle(output_file)
        
    print("Готово!")

In [ ]:
# Укажите свои пути и названия колонок
create_embeddings_in_batches(
    input_file='input_texts.csv',   # Ваш исходный файл
    output_file='output_with_emb.parquet', # Куда сохранить (лучше parquet)
    text_column='text',             # Имя колонки с текстом в CSV
    batch_size=128                  # Размер пачки (уменьшите, если не хватает памяти)
)

In [3]:
# Генерация тестовых данных
print("Создаём тестовые данные...")

# Создаём датасет с 200 примерами текстов
test_texts = [
    f"Это тестовый текст номер {i}. Он содержит различную информацию для демонстрации работы эмбеддингов." 
    for i in range(200)
]

# Добавляем разнообразные тексты
test_texts.extend([
    "Машинное обучение - это раздел искусственного интеллекта.",
    "Python - один из самых популярных языков программирования.",
    "Нейронные сети используются для решения сложных задач.",
    "Обработка естественного языка помогает компьютерам понимать текст.",
    "Трансформеры произвели революцию в NLP.",
])

# Создаём DataFrame
test_df = pd.DataFrame({
    'id': range(len(test_texts)),
    'text': test_texts,
    'category': ['test'] * len(test_texts)
})

# Сохраняем в CSV
test_df.to_csv('input_texts.csv', index=False)
print(f"Создан файл input_texts.csv с {len(test_texts)} строками")
print(test_df.head())

Создаём тестовые данные...
Создан файл input_texts.csv с 205 строками
   id                                               text category
0   0  Это тестовый текст номер 0. Он содержит различ...     test
1   1  Это тестовый текст номер 1. Он содержит различ...     test
2   2  Это тестовый текст номер 2. Он содержит различ...     test
3   3  Это тестовый текст номер 3. Он содержит различ...     test
4   4  Это тестовый текст номер 4. Он содержит различ...     test


In [7]:
# Теперь запускаем функцию генерации эмбеддингов
create_embeddings_in_batches(
    input_file='input_texts.csv',
    output_file='output_with_emb.parquet',
    text_column='text',
    batch_size=64
)

Используется устройство: mps
Загрузка модели all-MiniLM-L6-v2...
Чтение CSV файла...
Начинаем генерацию эмбеддингов (всего строк: 205, размер батча: 64)...


100%|██████████| 4/4 [00:00<00:00, 20.54it/s]

Сохранение в Parquet (рекомендуется)...


Готово!


In [5]:
# Проверяем результат
print("\n=== Проверка результата ===")
result_df = pd.read_parquet('output_with_emb.parquet')

print(f"Количество строк: {len(result_df)}")
print(f"Колонки: {result_df.columns.tolist()}")
print(f"\nПример первой строки:")
print(result_df.iloc[0][['id', 'text', 'category']])
print(f"\nРазмер эмбеддинга: {len(result_df.iloc[0]['embedding'])}")
print(f"Первые 10 значений эмбеддинга: {result_df.iloc[0]['embedding'][:10]}")


=== Проверка результата ===
Количество строк: 205
Колонки: ['id', 'text', 'category', 'embedding']

Пример первой строки:
id                                                          0
text        Это тестовый текст номер 0. Он содержит различ...
category                                                 test
Name: 0, dtype: object

Размер эмбеддинга: 384
Первые 10 значений эмбеддинга: [ 0.04614211  0.06408302 -0.01851688 -0.00331769 -0.07265981  0.0204317
  0.08442818  0.09363926 -0.02222545 -0.00988434]
